In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
"""
Llama 3.1 8B - LoRA ile Supervised Fine-Tuning
JSON çıktı üretimi için özelleştirilmiş eğitim scripti
Hugging Face Hub'a otomatik push ile
"""

import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from huggingface_hub import login, HfApi
import os
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')
SYSTEM_PROMPT = """Sen bir proje yönetimi asistanısın. Görevin, verilen Epic ve Task bilgilerini Jira API formatına çevirmek.

ÖNEMLİ KURALLAR:
1. Çıktı formatı SADECE JSON olmalı, başka hiçbir metin ekleme
2. Her task için "fields" objesi doldur
3. "parent.key" alanına Epic key'ini yaz (Epic başlığından otomatik oluştur: "Görev Ekleme ve Analizi" -> "GOREV-EKLEME-VE-ANALIZI")
4. "description" alanı mutlaka Jira'nın doc formatında olmalı
5. Öncelik çevirisi: Yüksek=High, Orta=Medium, Düşük=Low
6. "duedate" formatı: YYYY-MM-DD (Türkçe tarihten çevir: "15 Kasım 2025" -> "2025-11-15")
7. "timetracking.originalEstimate" formatı: "X gün" -> "Xd", "X saat" -> "Xh", "X hafta" -> "Xw"
8. "assignee" için hem name hem de accountId ekle. AccountId parantez içindeki employee_id değeridir (örn: "Ali Yılmaz (e14)" -> accountId: "e14")
9. "labels" etiketlerden oluştur
10. "summary" task başlığından al
11. JSON array'inin dışına hiçbir şey ekleme. Json biter bitmez çıktı üretmeyi kes

ÇIKTI FORMATI:
{
  "tasks": [
    {
      "epic_name": "Epic başlığı",
      "fields": {
        "project": {"key": "PROJE_KODU"},
        "issuetype": {"name": "Task"},
        "summary": "Task başlığı",
        "description": {
          "type": "doc",
          "version": 1,
          "content": [
            {
              "type": "paragraph",
              "content": [
                {"type": "text", "text": "Açıklama metni"}
              ]
            }
          ]
        },
        "assignee": {
          "name": "Ali Yılmaz",
          "accountId": "e14"
        },
        "priority": {"name": "High"},
        "duedate": "2025-11-15",
        "labels": ["etiket1", "etiket2"],
        "parent": {"key": "EPIC-KEY"},
        "timetracking": {"originalEstimate": "2d"}
      }
    }
  ]
}"""

# ============================================================================
# 1. KONFIGURASYON
# ============================================================================

class Config:
    # Hugging Face parametreleri
    HF_TOKEN = ""
    HF_REPO_NAME = "Berkesule/llama3.1_8b_sft_cosmos_kodllama"
    PUSH_TO_HUB = True

    # Model parametreleri
    MODEL_NAME = "ytu-ce-cosmos/Turkish-Llama-8b-Instruct-v0.1"

    # LoRA parametreleri (A100 80GB için optimize)
    LORA_R = 4  # Daha yüksek rank
    LORA_ALPHA = 8  # Alpha artırıldı
    LORA_DROPOUT = 0.05
    LORA_TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

    # Quantization kapalı (A100 için)
    USE_4BIT = False

    # Eğitim parametreleri (A100 80GB optimize)
    OUTPUT_DIR = "/content/drive/MyDrive/llama"
    NUM_TRAIN_EPOCHS = 3
    PER_DEVICE_TRAIN_BATCH_SIZE = 1  # Güvenli başlangıç
    GRADIENT_ACCUMULATION_STEPS = 2  # Effective batch = 16
    LEARNING_RATE = 2e-4
    MAX_GRAD_NORM = 0.3
    WARMUP_RATIO = 0.03
    LR_SCHEDULER_TYPE = "cosine"

    # Logging ve kaydetme
    LOGGING_STEPS = 10
    SAVE_STEPS = 100
    SAVE_TOTAL_LIMIT = 3

    # Token limitleri
    MAX_LENGTH = 6400  # Llama 3.1 için maksimum

    # Veri yolu
    JSON_DATA_PATH = "/content/drive/MyDrive/sentetik_data_stage1/traindata_20251025_145808.json"


# ============================================================================
# 2. HUGGING FACE LOGIN
# ============================================================================

def setup_huggingface():
    """Hugging Face'e login yap"""
    print("\n🔐 Hugging Face login yapılıyor...")

    token = Config.HF_TOKEN or os.environ.get("HF_TOKEN")

    if not token:
        print("⚠️  HF_TOKEN bulunamadı!")
        print("Lütfen token'ınızı girin (https://huggingface.co/settings/tokens):")
        token = input("Token: ").strip()

    try:
        login(token=token, add_to_git_credential=True)
        print("✅ Hugging Face login başarılı!")

        api = HfApi()
        user_info = api.whoami(token=token)
        username = user_info['name']
        print(f"👤 Kullanıcı: {username}")

        return token
    except Exception as e:
        print(f"❌ Login hatası: {str(e)}")
        raise


# ============================================================================
# 3. VERİ HAZIRLAMA
# ============================================================================

def load_json_data(file_path: str) -> List[Dict]:
    """JSON verisini yükle"""
    print(f"📂 Veri yükleniyor: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"✅ {len(data)} kayıt yüklendi")
    return data


def create_prompt(input_text: str, output_json: Dict) -> str:
    """
    Llama 3.1 Instruct formatında prompt oluştur
    Model, JSON çıktısını doğru formatta üretmeyi öğrenecek
    """
    output_str = json.dumps(output_json, ensure_ascii=False, indent=2)

    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

<|eot_id|><|start_header_id|>user<|end_header_id|>

{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{output_str}<|eot_id|>"""

    return prompt


def prepare_dataset(data: List[Dict], tokenizer) -> Dataset:
    """Dataset'i hazırla ve tokenize et"""
    print("\n🔧 Dataset hazırlanıyor...")

    formatted_data = []
    skipped_long = 0  # ✅ Değişken tanımlandı

    for idx, item in enumerate(data):
        try:
            input_text = item.get('input', '')
            output_json = item.get('output', {})

            if not input_text or not output_json:
                print(f"⚠️  {idx}. kayıt atlanıyor (eksik veri)")
                continue

            full_text = create_prompt(input_text, output_json)

            # Tokenize et - TRUNCATE YAPMA
            tokenized = tokenizer(
                full_text,
                truncation=False,  # ✅ Kesme
                padding=False,     # ✅ Padding yapma
                return_tensors=None,
                add_special_tokens=True
            )

            # Uzunluk kontrolü
            token_length = len(tokenized['input_ids'])
            if token_length > Config.MAX_LENGTH:
                skipped_long += 1
                if skipped_long <= 5:
                    print(f"⚠️  {idx}. kayıt atlandı: {token_length} token (max: {Config.MAX_LENGTH})")
                continue

            # ✅ Labels'ı kaldırdık, collator ekleyecek
            formatted_data.append({
                'input_ids': tokenized['input_ids'],
                'attention_mask': tokenized['attention_mask'],
            })

            if (idx + 1) % 50 == 0:
                print(f"  ✓ {idx + 1}/{len(data)} kayıt işlendi")

        except Exception as e:
            print(f"⚠️  {idx}. kayıtta hata: {str(e)}")
            continue

    if skipped_long > 0:
        print(f"\n⚠️  TOPLAM {skipped_long} kayıt çok uzun olduğu için atlandı")
        print(f"💡 MAX_LENGTH'i artırabilirsiniz (şu an: {Config.MAX_LENGTH})")

    print(f"\n✅ {len(formatted_data)} kayıt başarıyla hazırlandı")

    dataset = Dataset.from_list(formatted_data)
    return dataset


# ============================================================================
# 4. MODEL HAZIRLAMA
# ============================================================================

def load_model_and_tokenizer():
    """Model ve tokenizer'ı yükle, LoRA konfigürasyonu ile hazırla"""
    print("\n🤖 Model ve tokenizer yükleniyor...")

    tokenizer = AutoTokenizer.from_pretrained(
        Config.MODEL_NAME,
        trust_remote_code=True,
        padding_side="right",
        add_eos_token=True,
        add_bos_token=True
    )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    # Full Precision (BF16) - Quantization YOK
    model = AutoModelForCausalLM.from_pretrained(
        Config.MODEL_NAME,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
    )

    model.config.use_cache = False
    model.config.pretraining_tp = 1

    # LoRA konfigürasyonu
    peft_config = LoraConfig(
        r=Config.LORA_R,
        lora_alpha=Config.LORA_ALPHA,
        lora_dropout=Config.LORA_DROPOUT,
        target_modules=Config.LORA_TARGET_MODULES,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

    print("✅ Model hazır!")
    return model, tokenizer


# ============================================================================
# 5. EĞİTİM
# ============================================================================

def train():
    """Ana eğitim fonksiyonu"""
    print("=" * 80)
    print("🚀 LLAMA 3.1 8B - LoRA SFT EĞİTİMİ BAŞLIYOR")
    print("=" * 80)

    if Config.PUSH_TO_HUB:
        setup_huggingface()

    data = load_json_data(Config.JSON_DATA_PATH)
    model, tokenizer = load_model_and_tokenizer()
    train_dataset = prepare_dataset(data, tokenizer)

    # ✅ CUSTOM DATA COLLATOR - Dinamik Padding
    def custom_data_collator(features):
        """
        Farklı uzunluktaki sequence'leri batch'le
        - Dinamik padding (batch içinde en uzun'a göre)
        - Labels otomatik oluştur
        """
        batch = {}
        max_length = max(len(f['input_ids']) for f in features)

        input_ids_batch = []
        attention_mask_batch = []
        labels_batch = []

        for f in features:
            curr_len = len(f['input_ids'])
            pad_len = max_length - curr_len

            # Input IDs + padding
            padded_input_ids = f['input_ids'] + [tokenizer.pad_token_id] * pad_len
            input_ids_batch.append(padded_input_ids)

            # Attention mask (1=gerçek, 0=padding)
            padded_attention = f['attention_mask'] + [0] * pad_len
            attention_mask_batch.append(padded_attention)

            # Labels (padding=-100, loss'a katılmaz)
            padded_labels = f['input_ids'] + [-100] * pad_len
            labels_batch.append(padded_labels)

        batch['input_ids'] = torch.tensor(input_ids_batch, dtype=torch.long)
        batch['attention_mask'] = torch.tensor(attention_mask_batch, dtype=torch.long)
        batch['labels'] = torch.tensor(labels_batch, dtype=torch.long)

        return batch

    data_collator = custom_data_collator

    # Training arguments
    training_args = TrainingArguments(
        output_dir=Config.OUTPUT_DIR,
        num_train_epochs=Config.NUM_TRAIN_EPOCHS,
        per_device_train_batch_size=Config.PER_DEVICE_TRAIN_BATCH_SIZE,
        gradient_accumulation_steps=Config.GRADIENT_ACCUMULATION_STEPS,
        learning_rate=Config.LEARNING_RATE,
        max_grad_norm=Config.MAX_GRAD_NORM,
        warmup_ratio=Config.WARMUP_RATIO,
        lr_scheduler_type=Config.LR_SCHEDULER_TYPE,
        logging_steps=Config.LOGGING_STEPS,
        save_steps=Config.SAVE_STEPS,
        save_total_limit=Config.SAVE_TOTAL_LIMIT,
        bf16=True,
        optim="adamw_torch",
        logging_dir=f"{Config.OUTPUT_DIR}/logs",
        report_to="tensorboard",
        save_strategy="steps",
        group_by_length=True,
        ddp_find_unused_parameters=False,
        push_to_hub=Config.PUSH_TO_HUB,
        hub_model_id=Config.HF_REPO_NAME,
        hub_strategy="every_save",
        hub_private_repo=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=data_collator,
    )

    print("\n" + "=" * 80)
    print("🎯 EĞİTİM BAŞLADI")
    print("=" * 80 + "\n")

    trainer.train()

    print("\n💾 Model kaydediliyor...")
    trainer.save_model(Config.OUTPUT_DIR)
    tokenizer.save_pretrained(Config.OUTPUT_DIR)

    if Config.PUSH_TO_HUB:
        print("\n🚀 Model Hugging Face Hub'a push ediliyor...")
        try:
            trainer.push_to_hub(commit_message="Training completed - Final model")
            print(f"✅ Model başarıyla push edildi: https://huggingface.co/{Config.HF_REPO_NAME}")
        except Exception as e:
            print(f"⚠️  Push hatası: {str(e)}")

    print("\n" + "=" * 80)
    print("✅ EĞİTİM TAMAMLANDI!")
    print(f"📁 Model kaydedildi: {Config.OUTPUT_DIR}")
    if Config.PUSH_TO_HUB:
        print(f"🌐 Hugging Face: https://huggingface.co/{Config.HF_REPO_NAME}")
    print("=" * 80)


# ============================================================================
# 6. INFERENCE (TEST)
# ============================================================================

def test_model(input_text: str, model_path: str = None):
    """Eğitilmiş modeli test et"""
    if model_path is None:
        model_path = Config.OUTPUT_DIR

    print("\n🧪 Model test ediliyor...")

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )

    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM_PROMPT}<|eot_id|><|start_header_id|>user<|end_header_id|>

{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)

    if "<|start_header_id|>assistant<|end_header_id|>" in response:
        assistant_response = response.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
        assistant_response = assistant_response.replace("<|eot_id|>", "").strip()
    else:
        assistant_response = response

    print("\n" + "=" * 80)
    print("📤 MODEL ÇIKTISI:")
    print("=" * 80)
    print(assistant_response)
    print("=" * 80)

    return assistant_response


# ============================================================================
# 7. MAIN
# ============================================================================

if __name__ == "__main__":
    train()

    # Test örneği (isteğe bağlı)
    # test_input = "Proje: Web Sitesi Geliştirme, Ekip: 5 kişi, Süre: 3 ay"
    # test_model(test_input)

🚀 LLAMA 3.1 8B - LoRA SFT EĞİTİMİ BAŞLIYOR

🔐 Hugging Face login yapılıyor...
✅ Hugging Face login başarılı!
👤 Kullanıcı: Berkesule
📂 Veri yükleniyor: /content/drive/MyDrive/sentetik_data_stage1/traindata_20251025_145808.json
✅ 300 kayıt yüklendi

🤖 Model ve tokenizer yükleniyor...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

...

  ...08982.5d92194922ec.3063.0: 100%|##########| 5.09kB / 5.09kB            

  ...09629.ca72f77a398a.3046.0: 100%|##########|  88.0B /  88.0B            

  ...10017.1f3f44791d28.6890.0: 100%|##########|  88.0B /  88.0B            

  ...10177.03956bca2913.2389.0: 100%|##########| 8.78kB / 8.78kB            

✅ Model başarıyla push edildi: https://huggingface.co/Berkesule/llama3.1_8b_sft_cosmos_kodllama

✅ EĞİTİM TAMAMLANDI!
📁 Model kaydedildi: /content/drive/MyDrive/llama
🌐 Hugging Face: https://huggingface.co/Berkesule/llama3.1_8b_sft_cosmos_kodllama


GPU yetmediği için tam uzunlukta eğitimi yapamadık. Kısa veri üretilip yapılabilir max 4096 lenght

In [1]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `push` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credenti

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import os

# 1. Ana (Base) Model
BASE_MODEL_NAME = "ytu-ce-cosmos/Turkish-Llama-8b-Instruct-v0.1"

# 2. Sizin LoRA Adaptörünüz
ADAPTER_NAME = "Berkesule/kodllama_sft_cosmosllama"

# 3. Yüklenecek Yeni Birleşik Model Reposu
MERGED_REPO_NAME = "Berkesule/kodllama_sft_cosmosllama_merged"

print(f"Ana model yükleniyor: {BASE_MODEL_NAME}...")
# bfloat16, Llama 3 ve modern modeller için önerilen formattır
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map="auto", # Modeli GPU'ya otomatik yerleştirir
    trust_remote_code=True
)

print(f"Tokenizer yükleniyor: {BASE_MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_NAME,
    trust_remote_code=True
)

print(f"Adaptör yükleniyor: {ADAPTER_NAME}...")
# Adaptörü ana modelin üzerine yüklüyoruz
model = PeftModel.from_pretrained(base_model, ADAPTER_NAME)

print("Modeller birleştiriliyor (merge and unload)...")
# Adaptörü ana modelle kalıcı olarak birleştir
model = model.merge_and_unload()

print(f"Birleştirilmiş model Hugging Face Hub'a push ediliyor...")
print(f"Repo: {MERGED_REPO_NAME}")

# Modeli ve tokenizer'ı yeni reponuza yükleyin
# 'private=False' yaparak reponun public (herkese açık) olmasını sağlıyoruz
model.push_to_hub(MERGED_REPO_NAME, private=False, commit_message="Merged LoRA adapter")
tokenizer.push_to_hub(MERGED_REPO_NAME, private=False, commit_message="Added tokenizer")

print(f"✅ İşlem tamam! Model '{MERGED_REPO_NAME}' adresine yüklendi.")
print("Artık bu modeli vLLM ile direkt olarak yükleyebilirsiniz.")

Ana model yükleniyor: ytu-ce-cosmos/Turkish-Llama-8b-Instruct-v0.1...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

  ...0003-of-00004.safetensors:   0%|          | 4.89MB / 4.92GB            

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpc_oz1ulg/tokenizer.json:  89%|########9 | 15.3MB / 17.2MB            

✅ İşlem tamam! Model 'Berkesule/kodllama_sft_cosmosllama_merged' adresine yüklendi.
Artık bu modeli vLLM ile direkt olarak yükleyebilirsiniz.
